In [9]:
import requests
import csv
from tqdm import tqdm

#API endpoint
url = "https://blue-api.morpho.org/graphql"

# GraphQL query
query_template = """
query fetchMarketData($skip: Int, $first: Int) {
  markets(first: $first, skip: $skip) {
    items {
      uniqueKey
      lltv
      oracleAddress
      irmAddress
      loanAsset {
        address
        symbol
        decimals
      }
      collateralAsset {
        address
        symbol
        decimals
      }
      supplyingVaults {
        name
        address
      }
      state {
        borrowApy
        borrowAssets
        borrowAssetsUsd
        supplyApy
        supplyAssets
        supplyAssetsUsd
        fee
        utilization
        timestamp
      }
    }
  }
}
"""

#fetch data from the API
def fetch_data(skip, first):
    headers = {
        "Content-Type": "application/json"
    }
    payload = {
        "query": query_template,
        "variables": {
            "skip": skip,
            "first": first
        }
    }

    try:
        response = requests.post(url, json=payload, headers=headers)

        if response.status_code != 200:
            print(f"Error: Query failed with status code {response.status_code}")
            print(response.text)
            return None

        data = response.json()

        if 'errors' in data:
            print(f"GraphQL Error: {data['errors']}")
            return None

        return data.get('data', None)

    except Exception as e:
        print(f"Error during data fetch: {e}")
        return None


def save_data_to_csv(data, filename="morpho_vault_data.csv"):
    if data and 'markets' in data and 'items' in data['markets']:
        markets = data['markets']['items']

        with open(filename, mode='a', newline='') as file:
            writer = csv.writer(file)

            for market in markets:
                unique_key = market.get('uniqueKey', '')
                loan_asset = market.get('loanAsset', {}) or {}
                collateral_asset = market.get('collateralAsset', {}) or {}
                supplying_vaults = market.get('supplyingVaults', [{}])
                state = market.get('state', {}) or {}

                for vault in supplying_vaults:
                    row = [
                        unique_key,
                        loan_asset.get('symbol', ''),
                        loan_asset.get('address', ''),
                        collateral_asset.get('symbol', ''),
                        collateral_asset.get('address', ''),
                        vault.get('name', ''),
                        vault.get('address', ''),
                        market.get('oracleAddress', ''),
                        state.get('borrowApy', ''),
                        state.get('supplyApy', ''),
                        state.get('borrowAssets', ''),
                        state.get('borrowAssetsUsd', ''),
                        state.get('supplyAssets', ''),
                        state.get('supplyAssetsUsd', ''),
                        state.get('fee', ''),
                        state.get('utilization', ''),
                        state.get('timestamp', '')
                    ]
                    writer.writerow(row)

# Function to fetch and save the last 500 records.. as of 10/19/2024 the API only returns 455 asynchronous records
def fetch_last_500_records(batch_size=100, total_records=500, filename="morpho_vault_data.csv"):
    
    # Define the header for the CSV
    header = [
        'uniqueKey', 'loanAsset_symbol', 'loanAsset_address', 'collateralAsset_symbol', 
        'collateralAsset_address', 'vault_name', 'vault_address', 'oracleAddress',
        'borrowApy', 'supplyApy', 'borrowAssets', 'borrowAssetsUsd', 'supplyAssets', 
        'supplyAssetsUsd', 'fee', 'utilization', 'timestamp'
    ]

    
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)

    
    with tqdm(total=total_records, unit=" records", desc="Fetching records") as pbar:
        for skip in range(0, total_records, batch_size):
            data = fetch_data(skip, batch_size)
            if data:
                save_data_to_csv(data, filename)
                pbar.update(batch_size)
            else:
                print(f"Failed to fetch data for skip: {skip}")
                break

    print(f"Data fetching completed. Saved to {filename}")

fetch_last_500_records()


Fetching records: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:03<00:00, 145.91 records/s]

Data fetching completed. Saved to morpho_vault_data.csv
